In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score as r2_score

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchinfo import summary

from iTransformer import iTransformer

from tqdm import tqdm

from datetime import datetime
import os

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
file_path = "./data/wether.csv"

if os.path.exists(file_path):
    df = pd.read_csv(file_path, parse_dates=True, index_col=0)
else:
    url = "https://raw.githubusercontent.com/aweglteo/tokyo_weather_data/main/data.csv"
    df = pd.read_csv(url, parse_dates=True, index_col=0)
    df.to_csv(file_path)
df.info

<bound method DataFrame.info of             cloud  wind  ave_tmp  max_tmp  min_tmp  rain
2018-05-17    9.0   3.7     24.0     28.4     21.4   0.0
2018-05-18    7.8   3.0     23.4     27.1     20.0   0.0
2018-05-19    7.5   3.9     21.5     26.3     15.3   0.0
2018-05-20    4.3   3.5     16.8     22.1     13.1   0.0
2018-05-21   10.0   2.4     20.3     27.2     13.7   0.0
...           ...   ...      ...      ...      ...   ...
2023-05-13   10.0   3.0     16.7     18.1     15.4   5.0
2023-05-14   10.0   3.6     16.9     21.2     14.3   3.0
2023-05-15    9.8   2.0     16.1     18.1     14.6  15.5
2023-05-16    2.8   2.6     19.9     27.5     13.1   0.0
2023-05-17    0.0   2.3     22.8     31.6     15.1   0.0

[1827 rows x 6 columns]>

In [7]:
norm = lambda r: r / np.linalg.norm(r)
data = pd.DataFrame({col: norm(df[col]) for col in ['cloud', 'wind', 'ave_tmp', 'max_tmp', 'min_tmp', 'rain']})
data
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2018-05-17 to 2023-05-17
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cloud    1827 non-null   float64
 1   wind     1827 non-null   float64
 2   ave_tmp  1827 non-null   float64
 3   max_tmp  1827 non-null   float64
 4   min_tmp  1827 non-null   float64
 5   rain     1827 non-null   float64
dtypes: float64(6)
memory usage: 99.9 KB


In [ ]:
df_train, df_test = train_test_split(data, test_size=0.3, shuffle=False)

